## This notebook runs a RF on the whole MIC data (only train set of course) and save the 50 best features ranked by feature_importances

In [1]:
import sys
sys.path.remove('/home/pataki/.ipython')
sys.path.remove('/home/pataki/.local/lib/python3.6/site-packages')

In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_predict

%matplotlib inline

In [3]:
pd.__version__, np.__version__, sk.__version__

('0.25.1', '1.16.5', '0.21.2')

In [4]:
# need to set manually the location of these files
base = '/mnt/local/scratch/pataki/AMR_pred/final_processed/'

metaDF      = pd.read_csv(f'{base}meta.tsv', sep='\t')

micDF       = pd.read_csv(f'{base}ciprofloxacin_MICs.csv', dtype={'measurement':float})
zoneDF      = pd.read_csv(f'{base}ciprofloxacin_zone_diameters.csv', dtype={'measurement':float})
micDF  = micDF.append(zoneDF)

resfinderDF = pd.read_csv(f'{base}resfinder_results.csv')
snpDF       = pd.read_csv(f'{base}SNP_matrix.tsv', sep='\t', low_memory=False)
# need 64 GB RAM

In [5]:
mic_resfinderDF = pd.merge(micDF[['sample_alias', 'measurement_units', 'measurement']], 
                           resfinderDF, on='sample_alias', how='inner')

mic_resfinderDF = pd.merge(metaDF[['sample_alias', 'country']], 
                           mic_resfinderDF, on='sample_alias', how='inner')

allDF = pd.merge(mic_resfinderDF, snpDF, on='sample_alias', how='inner')

In [6]:
metaDF.shape, allDF.shape

((807, 8), (807, 836956))

we don't want to touch the test data, separate it!

In [7]:
trainDF = allDF[(~allDF.country.isnull()) & (allDF.measurement_units == 'mg/L')]
testDF  = allDF[(allDF.country.isnull())  & (allDF.measurement_units == 'mg/L')]

len(trainDF), len(testDF)

(438, 266)

In [8]:
trainDF.drop(['sample_alias', 'measurement_units'], axis=1, inplace=True)

/home/pataki/.conda/envs/AMR/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
trainDF.head()

,country,measurement,ARR-2,ARR-3,aac(3)-IIa,aac(3)-IId,aac(3)-IIe,aac(3)-IV,aac(3)-IVa,aac(3)-Id,...,CP009074.1_9656,CP009074.1_97,CP009074.1_9776,CP009074.1_9803,CP009074.1_9867,CP009074.1_9891,CP009074.1_9932,CP009074.1_994,CP009074.1_9950,CP009074.1_997
0,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Denmark,0.015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**save feature importances for each fold in a leave-one-country-out validation**

In [10]:
rf = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

importancesDF = pd.DataFrame()
predictions  = []
measurements = []

for i in pd.unique(trainDF.country.values):
    print(i)
    train_X = trainDF[trainDF.country != i]
    train_y = train_X.pop('measurement')
    train_X.pop('country')
    
    test_X = trainDF[trainDF.country == i]
    test_y = test_X.pop('measurement')
    test_X.pop('country')
    
    rf.fit(train_X, np.log2(train_y))
    impDF = pd.DataFrame({'imp':rf.feature_importances_, 'feat':train_X.columns})
    impDF['country'] = i
    importancesDF = importancesDF.append(impDF)
    
    predictions  = predictions + list(2**np.round(rf.predict(test_X)))
    measurements = measurements + list(test_y)

Denmark
Italy
United Kingdom
USA
Viet Nam


### Check feature importances for each fold and the aggregation

In [11]:
imp_pivot = pd.pivot_table(importancesDF[importancesDF.imp>0], 
                           index='feat', columns='country', values='imp', fill_value=0)
imp_pivot['sum'] = imp_pivot.sum(1)
imp_pivot.sort_values('sum', ascending=False).head(50).to_csv(f'{base}top50_important_features.csv')

In [12]:
imp_pivot.sort_values('sum', ascending=False).head(15).round(3)

country,Denmark,Italy,USA,United Kingdom,Viet Nam,sum
feat,,,,,,
CP009072.1_167295,0.555,0.567,0.575,0.558,0.011,2.265
CP009072.1_167306,0.115,0.148,0.114,0.141,0.690,1.209
CP009072.1_1589734,0.181,0.151,0.195,0.172,0.001,0.700
qnrS1,0.053,0.051,0.054,0.044,0.000,0.202
blaCTX-M-55,0.005,0.004,0.004,0.011,0.000,0.023
CP009072.1_3517597,0.000,0.001,0.000,0.000,0.011,0.012
CP009072.1_3517591,0.001,0.001,0.000,0.000,0.009,0.011
CP009072.1_1734215,0.001,0.001,0.000,0.001,0.009,0.011
CP009072.1_3517573,0.000,0.001,0.000,0.000,0.007,0.008


**Save the top50 features to a smaller table for easier/lighter further ML training.**

In [13]:
top50_feat = imp_pivot.sort_values('sum', ascending=False).head(50).index.tolist()

In [14]:
allDF[['sample_alias', 'country', 'measurement', 'measurement_units'] 
      + top50_feat].to_csv(f'{base}merged_top50.csv', index=False)